In [1]:
import pandas as pd
from tqdm import tqdm
from loguru import logger
import sys
import os
from datetime import timedelta
import gc

%load_ext autoreload
%autoreload 2
sys.path.append("/Users/xinc./Documents/GitHub/note")    # for mac
from module.get_info_Postgre import PostgresClient
from module.get_info_FinMind import FinMindClient
from module.plot_func import plot

# read

In [2]:
# 讀取資料庫中的 disposal_info 表格
database_name = "disposal"
client = PostgresClient(database = database_name)
disposal_info = client.fetch_table(table ='disposal_info', date_col='date', start_date='2018-01-01', end_date='2025-12-31', order_by='date')
disposal_info = disposal_info.loc[disposal_info["is_disposal"] == True]

In [3]:
disposal_info

,date,stock_id,is_disposal
121,2018-01-01,1475,True
289,2018-01-01,2424,True
1235,2018-01-01,3219,True
1493,2018-01-01,911616,True
1506,2018-01-01,8287,True
...,...,...,...
7641466,2025-11-11,2515,True
7641784,2025-11-11,8358,True
7641952,2025-11-11,6488,True
7642278,2025-11-11,3163,True


In [5]:
# [Log Suppression] 關閉 FinMind 的 loguru 輸出，避免洗版
logger.remove()  # 移除預設 handler
logger.add(sys.stderr, level="WARNING") # 只顯示警告以上的訊息

def fetch_and_filter_price(client, stock_id, event_dates, offset_days=3):
    """
    針對單檔股票，抓取處置日前後指定天數的股價資料。
    """
    # 決定抓取的時間範圍 (min - offset ~ max + offset)
    min_date = event_dates.min() - timedelta(days=offset_days)
    max_date = event_dates.max() + timedelta(days=offset_days)
    
    start_str = min_date.strftime('%Y-%m-%d')
    end_str = max_date.strftime('%Y-%m-%d')
    
    try:
        # 初始化並抓取資料
        client.initialize_frame(stock_id=stock_id, start_time=start_str, end_time=end_str)
        price_df = client.get_stock_price()
        
        if price_df.empty:
            return None
            
        price_df = price_df.reset_index()
        
        # 建立篩選遮罩 (Mask)
        # 只要日期落在任一處置事件的前後範圍內，即保留
        mask = pd.Series(False, index=price_df.index)
        for event_date in event_dates:
            d_start = event_date - timedelta(days=offset_days)
            d_end = event_date + timedelta(days=offset_days)
            # 這裡使用 pd.to_datetime 確保型別一致
            mask |= (price_df['Date'] >= pd.Timestamp(d_start)) & (price_df['Date'] <= pd.Timestamp(d_end))
            
        return price_df[mask].copy()
        
    except Exception as e:
        print(f"Error fetching {stock_id}: {e}")
        return None

# --- Main Execution ---

# 準備資料與 Client
disposal_events = disposal_info.copy()
unique_stocks = disposal_events['stock_id'].unique()
fm_client = FinMindClient()

print(f"Found {len(unique_stocks)} unique stocks with disposal events.")

all_prices = []
BATCH_SIZE = 50

# 迭代抓取
for i, stock_id in enumerate(tqdm(unique_stocks, desc="Fetching by Stock")):
    # 找出該股票所有處置日期
    stock_dates = disposal_events[disposal_events['stock_id'] == stock_id]['date']
    
    # 抓取並篩選
    result_df = fetch_and_filter_price(fm_client, stock_id, stock_dates)
    if result_df is not None:
        all_prices.append(result_df)
    
    # [Memory Optimization] 定期回收記憶體
    if i % BATCH_SIZE == 0:
        gc.collect()

# 最終整理
if all_prices:
    price_df = pd.concat(all_prices).drop_duplicates()
    print(f"Fetched {len(price_df)} price rows.")
    print(price_df.head())
else:
    print("No price data fetched.")

gc.collect()

Found 895 unique stocks with disposal events.


Fetching by Stock: 100%|██████████| 895/895 [02:50<00:00,  5.25it/s]

Fetched 33450 price rows.
        Date Stock_id   Open   High    Low  Close  Volume  TradingAmount
0 2017-12-29     1475  48.00  53.30  48.00  50.00  901560       45481144
1 2018-01-02     1475  48.20  48.20  45.00  45.00  169974        7779259
2 2018-01-03     1475  45.10  48.00  45.10  45.85   87200        4039709
3 2018-01-04     1475  46.95  47.95  46.00  46.80   65042        3053813
4 2018-01-05     1475  46.00  46.30  45.75  46.05   52010        2386158


0

## Analysis: Event Study on Disposal Effect

In [25]:
# --- Event Study Analysis (Corrected Gap Calculation) ---

analysis_frames = []

if 'price_df' in locals() and not price_df.empty:
    # 1. 準備股價表
    prices = price_df.sort_values(['Stock_id', 'Date']).copy()
    prices['Date'] = pd.to_datetime(prices['Date'])
    prices['trading_idx'] = prices.groupby('Stock_id').cumcount()
    
    # [Fix] 在這裡先計算 gap_days，避免合併後重複資料干擾
    prices['prev_trade_date'] = prices.groupby('Stock_id')['Date'].shift(1)
    prices['trade_date_diff'] = (prices['Date'] - prices['prev_trade_date']).dt.days
    prices['gap_days'] = prices['trade_date_diff'].fillna(1) - 1
    
    # 2. 準備事件表 (Start Date Detection)
    events = disposal_info[['stock_id', 'date']].sort_values(['stock_id', 'date'])
    events = events.rename(columns={'stock_id': 'Stock_id', 'date': 'event_date'})
    events['event_date'] = pd.to_datetime(events['event_date'])
    
    # 找出連續期間的起始日
    events['prev_date'] = events.groupby('Stock_id')['event_date'].shift(1)
    events['days_diff'] = (events['event_date'] - events['prev_date']).dt.days
    is_start = events['days_diff'].isna() | (events['days_diff'] > 1)
    start_events = events[is_start][['Stock_id', 'event_date']].copy()
    
    print(f"Original events: {len(events)}, Consolidated Start Events: {len(start_events)}")
    
    # 3. 找出事件日對應的交易日索引
    event_indices = pd.merge(
        start_events, 
        prices[['Stock_id', 'Date', 'trading_idx']], 
        left_on=['Stock_id', 'event_date'], 
        right_on=['Stock_id', 'Date'], 
        how='inner'
    )[['Stock_id', 'event_date', 'trading_idx']].rename(columns={'trading_idx': 'event_idx'})
    
    # 4. 合併股價與事件索引
    merged = pd.merge(prices, event_indices, on='Stock_id', how='inner')
    
    # 5. 計算相對天數 (Relative Trading Days)
    merged['relative_day'] = merged['trading_idx'] - merged['event_idx']
    
    # 6. 計算自然日相對天數 (Calendar Relative Day)
    merged['calendar_relative_day'] = (merged['Date'] - merged['event_date']).dt.days
    
    # 篩選範圍 (-3 ~ +3)
    offset_days = 3
    mask = (merged['relative_day'] >= -offset_days) & (merged['relative_day'] <= offset_days)
    event_study_df = merged[mask].copy()
    
    # 7. t_label
    def format_t(x):
        if x > 0: return f't+{x}'
        elif x < 0: return f't{x}'
        else: return 't+0'
    event_study_df['t_label'] = event_study_df['relative_day'].apply(format_t)
    
    # 8. 正規化 (Normalization)
    base_prices = event_study_df[event_study_df['relative_day'] == -1][['Stock_id', 'event_date', 'Close', 'Volume']]
    base_prices = base_prices.rename(columns={'Close': 'base_close', 'Volume': 'base_volume'})
    
    event_study_df = pd.merge(event_study_df, base_prices, on=['Stock_id', 'event_date'], how='left')
    event_study_df['norm_close'] = event_study_df['Close'] / event_study_df['base_close']
    event_study_df['norm_volume'] = event_study_df['Volume'] / event_study_df['base_volume']
    
    # 9. 整理欄位
    cols = ['Date', 'Stock_id', 't_label', 'relative_day', 'gap_days', 'calendar_relative_day', 'Open', 'High', 'Low', 'Close', 'Volume', 'norm_close', 'norm_volume']
    event_study_df = event_study_df[cols].sort_values(['Stock_id', 'Date'])
    
    print(f"Event Study Data Prepared: {len(event_study_df)} rows")
    print(event_study_df[['Date', 'Stock_id', 't_label', 'gap_days', 'calendar_relative_day', 'Close']].head(10))
else:
    print("price_df not found or empty.")


Original events: 36512, Consolidated Start Events: 2192
Event Study Data Prepared: 14272 rows
        Date Stock_id t_label  gap_days  calendar_relative_day  Close
0 2020-03-23   00642U     t-3       0.0                     -3  10.10
1 2020-03-24   00642U     t-2       0.0                     -2  10.36
2 2020-03-25   00642U     t-1       0.0                     -1  10.46
3 2020-03-26   00642U     t+0       0.0                      0  10.33
4 2020-03-27   00642U     t+1       0.0                      1  10.17
5 2020-03-30   00642U     t+2       2.0                      4   9.79
6 2020-03-31   00642U     t+3       0.0                      5  10.08
7 2020-03-13   00672L     t-1       0.0                     -3   4.36
8 2020-03-16   00672L     t+0       2.0                      0   4.03
9 2020-03-17   00672L     t+1       0.0                      1   3.41
